In [13]:
%pip install --user pymilvus "pymilvus[model]" sentence-transformers nltk FlagEmbedding ipywidgets
%pip install --user --upgrade jupyter ipywidgets

  Using cached FlagEmbedding-1.2.10-py3-none-any.whl
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl.metadata (32 kB)
Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
Using 


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install --user --upgrade jupyter_contrib_nbextensions notebook
!jupyter contrib nbextension install --user
!jupyter nbextension enable --py widgetsnbextension

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console contrib-nbextension dejavu events execute
kernel kernelspec lab labextension labhub migrate nbconvert
nbextensions_configurator notebook qtco

# фильтрация датасета

In [149]:
import pandas as pd
df = pd.read_excel("Вопросы и ответы.xlsx", header=1)
df = df.dropna()#[:5000]

In [63]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words('russian') + ["добрый", "день", "доброе", "утро", "вечер", "спасибо", "пожалуйста", "прошу", "коллеги", "уточнить"]
# stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\me\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
import regex as re
import numpy as np

df_clean = df.copy()
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.lower())
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'(\[.+\])|(\{.+\})', " ", regex=True))
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'[a-z-./]{4,}', " ", regex=True))
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'[^\w ]+', "", regex=True))
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'xc', "1с", regex=True))
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'xс', "1с", regex=True))
df_clean[['Описание', "Решение"]] = df_clean[["Описание", "Решение"]].apply(lambda row: row.str.replace(r'\s+', " ", regex=True))
df_clean['Описание'] = df_clean["Описание"].apply(lambda row: " ".join([x for x in row.split(" ") if x not in stops]))
df_clean[['Решение']] = df_clean[["Решение"]].apply(lambda row: row.str.replace(r'(добрый день)|(здравствуйте)|(добрый вечер)|(доброе утро)', "", regex=True))
df_clean[['Описание', "Решение"]].replace('', np.nan, inplace=True)

df_clean = df_clean.dropna()
df_clean["Решение"]

C:\Users\me\AppData\Local\Temp\ipykernel_20236\3270604459.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['Описание', "Решение"]].replace('', np.nan, inplace=True)


1         доработка функционала 1с erp xx возможна толь...
2        доработка функционала возможна в рамках запрос...
3         а в системе созданы отчеты по дебиторской и к...
4        доработка функционала возможна в рамках запрос...
5         наталья сергеевна доработка функционала возмо...
                               ...                        
22360     пользователь отключен от системы 1с erp при в...
22362     владимир олегович настроила для вас отчет вед...
22363     а очищен регистр рамочных договоров по догово...
22364                                   обращение отозвано
22365                                   обращение отозвано
Name: Решение, Length: 18670, dtype: object

# эмбеддинги

In [151]:
from pymilvus import model
from tqdm import tqdm

# embedding_fn = model.hybrid.BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

docs = df_clean["Описание"].str[:8192].values.tolist()
vectors = []# embedding_fn.encode_documents(tqdm(docs))
for doc in tqdm(docs):
    vectors.append(embedding_fn.encode_documents([doc])[0])
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

# Each entity has id, vector representation, raw text, and a subject label that we use
# to demo metadata filtering later.
data = [
    {"id": i, "vector": vectors[i], "text": df_clean.iloc[i]["Описание"], "answer": df_clean.iloc[i]["Решение"]}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 18670/18670 [55:34<00:00,  5.60it/s]  


Dim: 1024 (1024,)
Data has 18670 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'answer'])
Vector dim: 1024


In [10]:
print(data["Описание"].values.tolist())

['Коллеги, добрый день. Нет информации еще. Прошу уточнить, когда ждать решение по данному вопросу?    В Пн, XX/XX/XXXX в XX:XX +XXXX, Лушникова Ольга Владимировна пишет: Добрый день, Алексей.  Посмотрите, пожалуйста, письмо ниже. Очень ждем обратную связь.    {cid:imageXXX.png@XXDXXXXA.XXFAXXXX]  ', 'Добрый день, Елена Анатольевна!  Поясните, пожалуйста, по документу этап производства № АОXX-XXX.X.X от XX.XX.XXXX выбрана аналитика по сч. XX «Прочие расходы на персонал_XX_Приним», а в результате сумма затрат относится на ПР или делится по слоям налогового учета (№XXX.X.X от XX.XX.XXXX). Сумма XXX XXX,XX руб. – это премия при увольнении работника, по КШ принимаемая для НУ. Что нужно сделать, чтобы в налоговом учете было все в соответстии?  {cid:imageXXX.jpg@XXDAXXXE.DXXXBEEX]   ____________  ', 'Коллеги,  К закрытию декабря критичными являются следующие вопросы (в том числе, обозначенные аудиторами):  X.    Отчет по затратам (SDXXXXXXXX);  X.    Реклассификация договоров из не ДЦИ в ДЦИ

# векторная бд

In [ ]:
from pymilvus import MilvusClient

client = MilvusClient("http://docker.home:12532")

In [27]:
client.list_collections()

['demo_collection', 'guides', 'guides_collection']

In [156]:
client.drop_collection("demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=1024,  # The vectors we will use in this demo has 768 dimensions
)

In [157]:
for i in range(0, len(data), 100):
    res = client.insert(collection_name="demo_collection", data=data[i:i+100])

print(res)

{'insert_count': 70, 'ids': [18600, 18601, 18602, 18603, 18604, 18605, 18606, 18607, 18608, 18609, 18610, 18611, 18612, 18613, 18614, 18615, 18616, 18617, 18618, 18619, 18620, 18621, 18622, 18623, 18624, 18625, 18626, 18627, 18628, 18629, 18630, 18631, 18632, 18633, 18634, 18635, 18636, 18637, 18638, 18639, 18640, 18641, 18642, 18643, 18644, 18645, 18646, 18647, 18648, 18649, 18650, 18651, 18652, 18653, 18654, 18655, 18656, 18657, 18658, 18659, 18660, 18661, 18662, 18663, 18664, 18665, 18666, 18667, 18668, 18669]}


# инференс

In [236]:
def get_answers(query):
    query_vectors = embedding_fn.encode_queries([query])
    # If you don't have the embedding function you can use a fake vector to finish the demo:
    # query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]

    res = client.search(
        collection_name="demo_collection",  # target collection
        data=query_vectors,  # query vectors
        limit=15,  # number of returned entities
        output_fields=["text", "answer"],  # specifies fields to be returned
    )

    print("\n".join(map(lambda x: f"{x['distance']}: {x['entity']['answer']}", res[0])))
    answers = list(map(lambda x: x['entity']['answer'], res[0]))
    return answers

In [237]:
query = "Как добавить номенклатуру?"
answers = get_answers("query")

0.6708204746246338:  направлено письмо на эл почту документ от контрагента получен в формате который не утвержден фнс обращаем ваше внимание что в типовой форме соглашения об электронном документообороте приложение n xx к приказу госкорпорации росатом от xx июня г n п предусмотрены только форматы утвержденные фнс x организация x и организация x подтверждают взаимное согласие на выставление и получение следующих электронных документов счетовфактур корректировочных счетовфактур документов об отгрузке товаров выполнении работ оказании услуг передаче имущественных прав в том числе включающих в себя счетфактуру корректировочный счетфактуру составленных в электронной форме по форматам утвержденным действующими приказами фнс россии за исключением электронных документов составленных по форматам утвержденным приказами фнс россии от n ммв и n ммв исходя из этого документ полученный от контрагента не удовлетворяет условиям соглашения обработать такой документ в системе 1с erp нет возможности в да

In [238]:
from pymilvus import model
bge_rf = model.reranker.BGERerankFunction(
    model_name="BAAI/bge-reranker-v2-m3",  # Specify the model name. Defaults to `BAAI/bge-reranker-v2-m3`.
    device="cpu" # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

def rerank(query, answers):
    rank = bge_rf(query, answers)
    ranked_answers = list(map(lambda x: f"{round(x.score*1000)/10}%: {x.text}", [x for x in rank if x.score >= 0.005]))
    return ranked_answers

In [239]:
ranked_answers = rerank(query, answers)
print("\n".join(ranked_answers))

6.0%:  уважаемый алибек лемаевич проведена консультация по телефону для создания плана заявок с выбором в табличной части номенклатуры с типом товар необходимо идентифицировать номенклатуру в системе еос нси если позиция есть в 1с erp ей нужно присвоить gid соответствующий данной позиции в еос нси если номенклатура новая ее необходимо выгрузить из еос нси далее в 1с erp xx нужно дозаполнить данные в карточке номенклатуры и согласовать ее после того как карточка номенклатура будет проведена и утверждена необходимо создать заказ на внутреннее потребление блок закупки документы закупки все в заказе на внутреннее потребление нужно указать подразделениесклад если нужно направление деятельности действие в табличной части для позиции указать к обеспечению документ провести данный документ будет служить как назначение для закупаемой номенклатуры в плане закупок и всех последующих документах поступления закрытие согласовано 
3.9%:  ч по обращению для редактирования утвержденной карточки номенкл